In [ ]:
"""
Tree-of-Thought prompting
"""

In [ ]:
# define variables
NUM_FILES = 20
FOLDER_NAME = '../../determining_files_rename/random_samples_variables'
GPT_SAVED_FILE_NAME = 'rename_variable_gpt'

In [ ]:
# read in files from folder random_cells 
random_cells = []

# read in cells
for i in range(NUM_FILES):
    file_name = f'{FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

# read in readmes
with open(f'{FOLDER_NAME}/readmes.txt', 'r') as f:
    readmes = eval(f.read())

In [ ]:
main_prompt = """Rename the variable specified in the code delimited by triple backticks to a more meaningful name that reflects its usage and/or aligns with the project's purpose. Do not add, remove, or change anything else. First, suggest a new name. Then, rename the variable in the code. Structure your response under the following headings: 'New variable name' (the new variable name), 'Updated code' (the code with the variable renamed), and 'Explanation' (a 1-2 sentence explanation of the new variable name)."""

ex1_input = """Project purpose:
This project is about analyzing top movie genre trends from 2000 to 2010. We will look at the top genres and top movies for each year.

Variable to rename:
dat

Code:
```python
import pandas as pd

dat = pd.read_csv('data.csv')
dat.head()
```"""

ex1_output = """First, we look at how the identified variable is being used. 'dat' is being used to store the result of reading a csv file. We could rename it to 'data', which reflects its usage, however, we can do better. Looking at the project's purpose, a better name is 'movie_data_df' as the project is related to analyzing movie genres.

New variable name:
movie_data_df

Now, we rename the variable in the code. We need to update it in two places: the variable definition and the variable usage. We output the full code below with the variable renamed.

Updated code:
```python
import pandas as pd

movie_data_df = pd.read_csv('data.csv')
movie_data_df.head()
```

Now let's briefly explain the new variable name to the reader.

Explanation:
The new variable name 'movie_data_df' reflects its usage as a dataframe containing movie data."""

ex2_input = """Project purpose:
The focus of this project is determining the most common crime types in LA.

Variable to rename:
group

Code:
```python
group = df.groupby('gender')

for gender, grouped_data in group:
    print(f"Gender: {gender}")
    print(grouped_data['crime_type'].value_counts())
    print("\n")
```"""

ex2_output = """First, we look at how the identified variable is being used. 'group' is being used to group a dataframe by gender and is then later used in a print statement. Since the project is about crime types (and we are grouping by gender), a better name is 'gender_crime_groups' as it reflects its usage and aligns with the project's purpose.

New variable name:
gender_crime_groups

Now, we rename the variable in the code. We need to update it everywhere it is referenced. Here, that is its assignment and in the for-loop. We output the full code below with the variable renamed.

Updated code:
```python
gender_crime_groups = df.groupby('gender')

for gender, grouped_data in gender_crime_groups:
    print(f"Gender: {gender}")
    print(grouped_data['crime_type'].value_counts())
    print("\n")
```

Now let's briefly explain the new variable name to the reader.

Explanation:
The new variable name 'gender_crime_groups' aligns with its purpose of grouping crime data by gender."""

ex3_input = """Project purpose:
This repository contains a collection of graph theory assignments for MAT381.

Variable to rename:
queue

Code:
```python
queue[0] = queue[0].lower()
while queue:
    vertex = queue.popleft()
    print(vertex, end=' ')

    for neighbor in graph[vertex]:
        if neighbor not in visited:
            queue.append(neighbor)
            visited.add(neighbor)
```"""

ex3_output = """First, we look at how the identified variable is being used. 'queue' is being used to store a list of vertices. Since the project is about graph theory, a better name is 'vertex_queue' as it reflects its usage and aligns with the project's purpose.

New variable name:
vertex_queue

Now, we rename the variable in the code. We need to change it where it is being updated in the first line and used in the while-loop. We output the full code below with the variable renamed.

Updated code:
```python
vertex_queue[0] = vertex_queue[0].lower()
while vertex_queue:
    vertex = vertex_queue.popleft()
    print(vertex, end=' ')

    for neighbor in graph[vertex]:
        if neighbor not in visited:
            vertex_queue.append(neighbor)
            visited.add(neighbor)
```

Now let's briefly explain the new variable name to the reader.

Explanation:
By renaming queue to vertex_queue, it's now clearer that the variable is being used to store vertices."""

ex4_input = """Project purpose:
This project is for analyzing the relationship between the number of hours studied and the exam scores of students.

Variable to rename:
result

Code:
```python
result, info = pearsonr(queue['Hours'], queue['Scores'])
print(result)
```"""

ex4_output = """First, we look at how the identified variable is being used. 'result' is used in an unpacking statement to store the result of a correlation test. We are not interested in the other variable 'info' since it was not specified in the prompt. Since the project is about analyzing the relationship between hours studied and exam scores, a better name for 'result' is 'hours_scores_correlation' as it reflects its usage and aligns with the project's purpose.

New variable name:
hours_scores_correlation

Now, we rename the variable in the code. There are only two places to change it: in its definition and in a print statement. We output the full code below with the variable renamed.

Updated code:
```python
hours_scores_correlation, info = pearsonr(queue['Hours'], queue['Scores'])
print(hours_scores_correlation)
```

Now let's briefly explain the new variable name to the reader.

Explanation:
The variable name 'hours_scores_correlation' reflects the purpose of the variable, which is to store the correlation between the number of hours studied and the exam scores of students."""

identify_vote_prompt = """Given an original task and multiple choices, choose the best answer for the original task. Analyze each choice in detail, then conclude in the last line 'The best choice is {s}', where s is the integer id of the choice. If all choices are equally good, return the smallest id. If no choice is good, return 0.
Original task: Suggest a more meaningful name for the variable specified in the code delimited by triple backticks that reflects its usage and/or aligns with the project's purpose."""

identify_vote_ex1_input = """Project purpose:
This project is about analyzing top movie genre trends from 2000 to 2010. We will look at the top genres and top movies for each year.

Variable to rename:
dat

Code:
```python
import pandas as pd

dat = pd.read_csv('data.csv')
dat.head()
```

Choice 1:
data
Choice 2:
movie
Choice 3:
movie_df_data"""

identify_vote_ex1_output = """Choice 1 is not the best choice. 'data' is a very generic name and does not reflect the project's purpose.
Choice 2 is also not the best choice. While 'movie' hints at the project's purpose, it does not reflect the variable's usage.
Choice 3 is the best choice. 'movie_df_data' reflects the variable's usage and aligns with the project's purpose.

The best choice is 3."""

code_vote_prompt = """Given an original task and multiple choices, choose the best answer for the original task. Analyze each choice in detail, then conclude in the last line 'The best choice is {s}', where s is the integer id of the choice. If all choices are equally good, return the smallest id. If no choice is good, return 0.
Original task: Rename the variable specified in the code delimited by triple backticks to the new name specified. Update the variable name everywhere it is referenced."""

code_vote_ex1_input = """Rename the variable 'result' to 'hours_scores_correlation' in the code delimited by triple backticks. Do not change anything else.
Original code:
```python
result, info = pearsonr(queue['Hours'], queue['Scores'])
print(result)
```

Choice 1:
```python
hours_scores_correlation, info = pearsonr(queue['Hours'], queue['Scores'])
print(hours_scores_correlation)
```
Choice 2:
```python
hours_scores_correlation, info_correlation = pearsonr(queue['Hours'], queue['Scores'])
print(hours_scores_correlation)
```
Choice 3:
```python
result, info = pearsonr(queue['Hours'], queue['Scores'])
print(result)
```
"""

code_vote_ex1_output = """Choice 1 is the best choice. It renames the variable 'result' to 'hours_scores_correlation' in the code delimited by triple backticks. It does not change anything else.
Choice 2 is incorrect. While it correctly renames the variable 'result' to 'hours_scores_correlation', it also renames the variable 'info' to 'info_correlation'. This is not specified in the prompt.
Choice 3 is incorrect. It does not rename the variable 'result' to 'hours_scores_correlation'.

The best choice is 1."""

def get_cot_prompt(purpose, name, cell_src):
    return [
        {"role": "user", "content": main_prompt},
        {"role": "user", "content": ex1_input},
        {"role": "assistant", "content": ex1_output},
        {"role": "user", "content": ex2_input},
        {"role": "assistant", "content": ex2_output},
        {"role": "user", "content": ex3_input},
        {"role": "assistant", "content": ex3_output},
        {"role": "user", "content": ex4_input},
        {"role": "assistant", "content": ex4_output},
        {"role" : "user", "content" : f"Project purpose:\n{purpose}\n\nVariable to rename:\n{name}\n\nCode:\n```python\n{cell_src}\n```"},
    ]

def get_identified_names(identify_trials, identify_completions):
    identified_names = []
    for i in range(identify_trials):
        if identify_completions.choices[i].finish_reason == 'stop':
            try:
                name = identify_completions.choices[i]['message']['content'].split('New variable name:')[1].strip("\n").split("\n")[0]
            except:
                print("unexpected format for name", identify_completions.choices[i]['message']['content'])
                name = None
            else:
                if len(name.split('`')) == 3:
                    name = name.split('`')[1]
        else:
            name = None
        identified_names.append(name)
    return identified_names

def get_identify_vote_msgs(purpose, original_name, cell_src):
    def func(choices):
        final_msg = f"Project purpose:\n{purpose}\n\nVariable to rename:\n{original_name}\n\nCode:\n```python\n{cell_src}\n```\n\n"

        for i, choice in enumerate(choices):
            final_msg += f"Choice {i + 1}:\n{choice}\n"
        
        return [
        {"role": "user", "content": identify_vote_prompt},
        {"role": "user", "content": identify_vote_ex1_input},
        {"role": "assistant", "content": identify_vote_ex1_output},
        {"role": "user", "content": final_msg}
        ]
    
    return func

def get_code_vote_msgs(original_name, cell_src):
    def func(new_name, choices):
        final_msg = f"Rename the variable '{original_name}' to '{new_name}' in the code delimited by triple backticks. Do not change anything else.\nOriginal code:\n```python\n{cell_src}\n```\n\n"
        
        for i, choice in enumerate(choices):
            final_msg += f"Choice {i + 1}:\n{choice}\n"
        
        return [
        {"role": "user", "content": code_vote_prompt},
        {"role": "user", "content": code_vote_ex1_input},
        {"role": "assistant", "content": code_vote_ex1_output},
        {"role" : "user", "content" : final_msg}
        ]

    return func

In [ ]:
# GPT Tree of Thought
import sys
sys.path.append('../../../')
from tree_of_thought import solve_toc

identify_trials = 5
code_trials = 3
identify_vote_trials = 5
code_vote_trials = 5
identify_stop = "Updated code"

# identify and remove unused using GPT
gpt_results = []
for i in range(NUM_FILES):
    input_msgs_cot = get_cot_prompt(readmes[i], 'variable_def', random_cells[i])
    get_identified_names_func = get_identified_names
    get_identify_votes_msgs_func = get_identify_vote_msgs(readmes[i], 'variable_def', random_cells[i])
    get_code_votes_msgs_func = get_code_vote_msgs('variable_def', random_cells[i])

    print(f'Processing file {i}')
    identified, updated_code = solve_toc(input_msgs_cot, identify_trials, code_trials, identify_vote_trials, code_vote_trials, identify_stop, get_identified_names_func, get_identify_votes_msgs_func, get_code_votes_msgs_func)
    print(f'File {i} - {identified}')
    gpt_results.append({'identified': identified, 'updated_code': updated_code})

# save the results to a file
with open(GPT_SAVED_FILE_NAME, 'w') as f:
    f.write(str(gpt_results))

In [ ]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [ ]:
# save the results to a variable
gpt_new_names = [var['identified'] for var in gpt_results]
gpt_new_code = [var['updated_code'] for var in gpt_results]

In [ ]:
# Count the number of times the variable name is/isn't successfully changed
import sys
sys.path.append('../../determining_files_rename')
from ast_determine_usable_items import compare_code

pass_count = 0
fail_count = 0

for i in range(NUM_FILES):
    if compare_code(random_cells[i], gpt_new_code[i], 'variable_def', gpt_new_names[i]):
        pass_count += 1
    else:
        fail_count += 1
        print(f'Failed on file {i}')

print(f'Pass count: {pass_count}, {pass_count / (pass_count + fail_count) * 100}%')
print(f'Fail count: {fail_count}, {fail_count / (pass_count + fail_count) * 100}%')